### Function Computing  $\sigma_n(x_k)$

In [5]:
import numpy as np
a0, a1, a2, a3, a4, a5 = var('a0 a1 a2 a3 a4 a5')
b0, b1, b2 = var('b0 b1 b2')
def S_n(q, x, n):
    """
    S_n is a function that computes the power series sigma_n(x_k) written in the accompanying report.
    :param q: Input polynomial of any degree
    :param x: The variable in which the polynomial is in.
    :param n: The n, in sigma_n(x_k)
    :return: Returns sigma_n(x_k)
    """ 
    x = var('x') 
    deg = q.degree(x)
    coefficients = []

    for i in range(deg+1):
        coefficients.append(q.coefficient(x,i))
    S_n_values = [None] * (n)

    def compute_S_n(n):
        if n > deg:
            coeff1 = 0
        else:
            coeff1 = coefficients[deg - n]
        if S_n_values[n-1] is not None:
            return S_n_values[n-1]

        
        Sn = -n * coeff1
        for j in range(1, n + 1):
            if j > deg:
                coeff = 0
            else:
                coeff = coefficients[deg - j]
            Sn -= compute_S_n(n - j) * coeff
        S_n_values[n-1] = Sn.expand()
        return S_n_values[n-1]
    
    return compute_S_n(n)

### Example - $f(x) = x^5 + a_4x^4 + a_3x^3 + a_2x^2 + a_1x^1 + a_0$ - $\sigma_n(x_k)$ - $n = 1, 2, 3, 4, 5, 6$

In [6]:
q = x^5 + a4*x^4 + a3*x^3 + a2*x^2 + a1*x^1 + a0
print(S_n(q,x,6))
print(S_n(q,x,5))
print(S_n(q,x,4))
print(S_n(q,x,3))
print(S_n(q,x,2))
print(S_n(q,x,1))

a4^6 - 6*a3*a4^4 + 9*a3^2*a4^2 + 6*a2*a4^3 - 2*a3^3 - 12*a2*a3*a4 - 6*a1*a4^2 + 3*a2^2 + 6*a1*a3 + 6*a0*a4
-a4^5 + 5*a3*a4^3 - 5*a3^2*a4 - 5*a2*a4^2 + 5*a2*a3 + 5*a1*a4 - 5*a0
a4^4 - 4*a3*a4^2 + 2*a3^2 + 4*a2*a4 - 4*a1
-a4^3 + 3*a3*a4 - 3*a2
a4^2 - 2*a3
-a4


In [7]:
alpha,beta = var('alpha beta')
def SDictGen(f,q):
    """
    SDictGen is a function that generates a dictionary of the required substitutions from the expansion of the Tschirnhaus transformation polynomial.
    :param f: Tschirnhaus Transformation Polynomial (Quadratic for Pricipal, Quartic for Bring-Jerrard)
    :param q: The polynomial being expanded.
    :return: Returns subDict, a dictionary object.
    """ 
    subDict = {}
    for i in range(1,5*f.degree(x)+1):
        print(f'{i+1}/{5*f.degree(x)+1}')
        subDict[var(f'S{i}')] = S_n(q,x,i)
    return(subDict)
def expandSub(n, f, subDict):
    """
    expandSubs performs, sigma_n(y_k) = sum(f^n) for quintic equations. Due to the quirks of SAGE a term-by-term approach was required.
    :param n: The n in sigma_n(y_k)
    :param f: Tschirnhaus Transformation Polynomial (Quadratic for Pricipal, Quartic for Bring-Jerrard)
    :param subDict: a dictionary object generated by SDictGen function
    :return: Returns a dictionary object for substitution in terms of coefficients.
    """ 
    expanded_terms = []
    expand = sum(f^n for i in [1..5]).expand()
    deg=expand.degree(x)
    ops = expand.operands()
    collector=0
    for i, operand in enumerate(ops):
        deg_op = operand.degree(x)
        substitution_dict={}
        substitution_dict[x^deg_op] = var('S{}'.format(deg_op))/5
        sub = operand.subs(substitution_dict)
        collector+=sub 
    return(collector.subs(subDict))

def PrincipalForm(q,num:bool):
    """
    PrincipalForm performs a quadratic Tschirnhaus transformation on a general quintic q
    :param q: Input general quintic polynomial.
    :param num: Whether the output is in numeric or exact symbolic form.
    :return: Returns a transformed principal polynomial.
    """ 
    f = x^2+alpha*x+beta
    subDict=SDictGen(f,q)
    eq1 = expandSub(1,f,subDict)
    eq2 = expandSub(2,f,subDict)
    eq3 = expandSub(3,f,subDict)
    eq4 = expandSub(4,f,subDict)
    eq5 = expandSub(5,f,subDict)
    solutions = solve([eq1 == 0, eq2 == 0], [alpha, beta])
    solution=solutions[0]
    alphasol=solution[0].right()
    betasol=solution[1].right()
    eq3Sub = eq3.subs({alpha: alphasol, beta: betasol})
    eq4Sub = eq4.subs({alpha: alphasol, beta: betasol})
    eq5Sub = eq5.subs({alpha: alphasol, beta: betasol})
    b_2, b_1, b_0 = var('b_2 b_1 b_0')
    principalTerms_Coeff = solve([(eq3Sub) == -3*b_2,eq4Sub == -4*b_1,eq5Sub == -5*b_0], [b_2, b_1, b_0])[0]
    if num:
        p = x^5 + principalTerms_Coeff[0].right().n()*x^2 + principalTerms_Coeff[1].right().n()*x^1 + principalTerms_Coeff[2].right().n()
        print(f"Alpha (Quadratic) = {alphasol.n()}")
        print(f"Beta (Quadratic) = {betasol.n()}")
        return([p, principalTerms_Coeff[0].right().n(),principalTerms_Coeff[1].right().n(),principalTerms_Coeff[2].right().n()])
    else:
        p = x^5 + principalTerms_Coeff[0].right()*x^2 + principalTerms_Coeff[1].right()*x^1 + principalTerms_Coeff[2].right()
        print(f"Alpha (Quadratic) = {(alphasol)}")
        print(f"Beta (Quadratic) = {(betasol.full_simplify())}")
        return(p.expand())

### Example De Moivre' Quintic $f(x) = x^5 +0x^4 +5ax^3+0x^2+5*a^2x+b$


In [8]:
a,b=var('a b')
q = x^5 +0*x^4 +5*a*x^3+0*x^2+5*a^2*x+b
r = PrincipalForm(q,num=False)
print(r)

2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
Alpha (Quadratic) = -sqrt(a)
Beta (Quadratic) = 2*a
-22*a^5 + 15*a^4*x - 10*a^3*x^2 + x^5 + 9*a^(5/2)*b - 5*sqrt(a)*b*x^2 - b^2


### Example General Quintic $f(x) = x^5 +a_4 x^4 +a_3x^3+a_2x^2+a_1x+a_0$


In [9]:
q = x^5 + a4*x^4 + a3*x^3 + a2*x^2 + a1*x^1 + a0
r = PrincipalForm(q,num=False)
print(r)

2/11
3/11
4/11
5/11
6/11
7/11
8/11
9/11
10/11
11/11
Alpha (Quadratic) = 1/2*(4*a4^3 - 13*a3*a4 + 15*a2 - sqrt(40*a2*a4^3 + 60*a3^3 - 190*a2*a3*a4 - 5*(3*a3^2 - 16*a1)*a4^2 + 225*a2^2 - 200*a1*a3))/(2*a4^2 - 5*a3)
Beta (Quadratic) = 1/10*(5*a3*a4^2 - 20*a3^2 + 15*a2*a4 - sqrt(40*a2*a4^3 + 60*a3^3 - 190*a2*a3*a4 - 5*(3*a3^2 - 16*a1)*a4^2 + 225*a2^2 - 200*a1*a3)*a4)/(2*a4^2 - 5*a3)
3/5*a3^5*a4^10/(32*a4^10 - 400*a3*a4^8 + 2000*a3^2*a4^6 - 5000*a3^3*a4^4 + 6250*a3^4*a4^2 - 3125*a3^5) - 4*a2*a3^3*a4^11/(32*a4^10 - 400*a3*a4^8 + 2000*a3^2*a4^6 - 5000*a3^3*a4^4 + 6250*a3^4*a4^2 - 3125*a3^5) + 32/5*a2^2*a3*a4^12/(32*a4^10 - 400*a3*a4^8 + 2000*a3^2*a4^6 - 5000*a3^3*a4^4 + 6250*a3^4*a4^2 - 3125*a3^5) + 24/5*a1*a3^2*a4^12/(32*a4^10 - 400*a3*a4^8 + 2000*a3^2*a4^6 - 5000*a3^3*a4^4 + 6250*a3^4*a4^2 - 3125*a3^5) - 64/5*a1*a2*a4^13/(32*a4^10 - 400*a3*a4^8 + 2000*a3^2*a4^6 - 5000*a3^3*a4^4 + 6250*a3^4*a4^2 - 3125*a3^5) - 51/5*a3^6*a4^8/(32*a4^10 - 400*a3*a4^8 + 2000*a3^2*a4^6 - 5000*a3^3*a4^4 + 6250*a3

In [11]:
alpha, beta, gamma, delta = var('alpha beta gamma delta')
    
def BringJerrardForm(q,num:bool):
    """
    BringJerrardForm performs a quartic Tschirnhaus transformation on a principal quintic q. Can take approx minutes to compute.
    :param q: Input principal quintic polynomial.
    :param num: Whether the output is in numeric or exact symbolic form.
    :return: Returns a transformed bring-jerrard polynomial.
    """ 
    b2, b1, b0 = q.coefficient(x,0),q.coefficient(x,1),q.coefficient(x,2)
    print("Starting...")
    f = x^4+alpha*x^3+beta*x^2+gamma*x+delta
    subDict = SDictGen(f,q)
    eq1 = expandSub(1,f,subDict)
    eq2 = expandSub(2,f,subDict)
    eq3 = expandSub(3,f,subDict)
    eq4 = expandSub(4,f,subDict)
    eq5 = expandSub(5,f,subDict)
    print("Equations Expanded")
    deltasol = solve([eq1==0], [delta])[0].right()
    print("Solved: Delta")
    betaSub = -5*q.coefficient(x,0)/(3*q.coefficient(x,2))-4*q.coefficient(x,1)*alpha/(3*q.coefficient(x,2))
    print("Solved: Beta")
    alphasol_pre_pre = eq2.subs({delta:deltasol}).full_simplify()
    alphasol_pre = alphasol_pre_pre.subs({beta:betaSub})
    alphasol = solve([alphasol_pre == 0], [alpha])
    #Select Branch
    alphaSub=alphasol[0].right()
    print("Solved: Alpha")
    betaSub_2 = betaSub.subs({alpha:alphaSub})
    deltaSub = deltasol.subs({alpha:alphaSub})
    gammasol_pre = eq3.subs({delta:deltaSub,beta:betaSub_2,alpha:alphaSub})
    gammasol = solve([gammasol_pre==0],[gamma])
    #Select branch
    gammaSub=gammasol[2].right()
    print("Solved: Gamma")
    alphaSubFinal = alphaSub.subs({gamma:gammaSub})
    betaSubFinal = betaSub_2.subs({gamma:gammaSub})
    deltaSubFinal = deltaSub.subs({gamma:gammaSub})
    d1, d0 = var('d1 d0')
    BringTerms_Coeff = solve([eq4 == -4*d1, eq5 == -5*d0],[d1,d0])[0]
    BringTerms_Coeff_Sub1 = BringTerms_Coeff[0].subs({delta:deltaSubFinal,beta:betaSubFinal,alpha:alphaSubFinal, gamma:gammaSub})
    BringTerms_Coeff_Sub2 = BringTerms_Coeff[1].subs({delta:deltaSubFinal,beta:betaSubFinal,alpha:alphaSubFinal, gamma:gammaSub})
    BringTerms_Coeff_Sub = [BringTerms_Coeff_Sub1, BringTerms_Coeff_Sub2]
    if num:
        p = x^5 + BringTerms_Coeff_Sub[0].right().n()/BringTerms_Coeff_Sub[1].right().n() * x^1 + 1
    else:
        p = x^5 + BringTerms_Coeff_Sub[0].right()/BringTerms_Coeff_Sub[1].right() * x^1 + 1
    return(p)

### Example General Principal Quintic to Bring-Jerrard $f(x) = x^5 +0x^4 +0 x^3+a_2x^2+a_1x+a_0$


In [12]:
q = x^5 + 0*x^4 + 0*x^3+a2*x^2+a1*x+a0

s = (BringJerrardForm(q, False))
print(s)

Starting...
2/21
3/21
4/21
5/21
6/21
7/21
8/21
9/21
10/21
11/21
12/21
13/21
14/21
15/21
16/21
17/21
18/21
19/21
20/21
21/21
Equations Expanded
Solved: Delta
Solved: Beta
Solved: Alpha
Solved: Gamma
x^5 - 75/2*(500*(50*(1/50)^(2/3)*(6*(4374*a2^11 - 59859*a1^3*a2^7 + 205440*a1^6*a2^3 + 1040625*a0^4*a1*a2^3 + 1250000*a0^5*a1^2 + 1000*(81*a2^6 + 510*a1^3*a2^2 - 10*sqrt(-135*a1^2*a2^4 + 540*a0*a2^5 + 1280*a1^5 - 8000*a0*a1^3*a2 + 11250*a0^2*a1*a2^2 + 15625*a0^4)*a1^2)*a0^3 + 25*(37854*a1^2*a2^5 - 27520*a1^5*a2 + 189*sqrt(-135*a1^2*a2^4 + 540*a0*a2^5 + 1280*a1^5 - 8000*a0*a1^3*a2 + 11250*a0^2*a1*a2^2 + 15625*a0^4)*a1*a2^3)*a0^2 + 2*(62451*a1*a2^8 - 434880*a1^4*a2^4 + 51200*a1^7 + 81*sqrt(-135*a1^2*a2^4 + 540*a0*a2^5 + 1280*a1^5 - 8000*a0*a1^3*a2 + 11250*a0^2*a1*a2^2 + 15625*a0^4)*(3*a2^6 - 80*a1^3*a2^2))*a0 - (783*a1^2*a2^5 - 5120*a1^5*a2)*sqrt(-135*a1^2*a2^4 + 540*a0*a2^5 + 1280*a1^5 - 8000*a0*a1^3*a2 + 11250*a0^2*a1*a2^2 + 15625*a0^4))/(729*a2^9 - 8640*a1^3*a2^5 + 25600*a1^6*a2 + 90000*a0^

### Example Numerical Principal Quintic to Bring-Jerrard $f(x) = x^5 +0x^4 +0 x^3+1x^2+1x+2$


In [13]:
q= x^5 + 0*x^4 + 0*x^3+1*x^2+1*x+2

s = (BringJerrardForm(q,True))
print(s)

Starting...
2/21
3/21
4/21
5/21
6/21
7/21
8/21
9/21
10/21
11/21
12/21
13/21
14/21
15/21
16/21
17/21
18/21
19/21
20/21
21/21
Equations Expanded
Solved: Delta
Solved: Beta
Solved: Alpha
Solved: Gamma
x^5 + 0.0193518281123430*x + 1
